In [1]:
from pathlib import Path
import re
import pandas as pd

folder = Path("results")
metric_files = folder.glob("*_test_metrics.csv")

pattern = re.compile(r"""
    (?P<models>Signature_FULL)_ # models
    dp(?P<dp>\d+)_             # dp
    sl(?P<sl>\d+)_             # sl
    pl(?P<pl>\d+)_             # pl
    dm(?P<dm>\d+)_             # dm
    nh(?P<nh>\d+)_             # nh
    nl(?P<nl>\d+)_             # nl
    si(?P<si>\d+)_             # si
    sc(?P<sc>\d+)_             # sc
    hc(?P<hc>\d+)_             # hc
    mp(?P<mp>\d+)_             # mp
    ff(?P<ff>\d+)_             # ff
    deExp_tp(?P<tp>[\d\.]+?)_   # tp (temperature)
    cb(?P<cb>[\d\.]+?)_         ## 변경된 부분: cb 파라미터 추가
    (?P<itr>\d+)_test_metrics\.csv$ # itr
""", re.VERBOSE)

dfs = []
for f in metric_files:
    m = pattern.search(f.name)
    if not m:
        print(f"pass: {f.name}")
        continue

    meta = m.groupdict()

    int_fields = ["dp", "sl", "pl", "dm", "nh", "nl",
                  "si", "sc", "hc", "mp", "ff", "itr"]
    for k in int_fields:
        if meta.get(k):
            meta[k] = int(meta[k])

    float_fields = ['tp', 'cb'] 
    for k in float_fields:
        if meta.get(k):
            meta[k] = float(meta[k])

    df = pd.read_csv(f)

    meta_cols = ["models", "dp", "sl", "pl", "dm", "nh",
                 "nl", "si", "sc", "hc", "mp", "ff", "tp", "cb", "itr"] 
    for col in meta_cols:
        df[col] = meta[col]

    dfs.append(df)

if not dfs:
    print("error")
else:
    combined_df = pd.concat(dfs, ignore_index=True)

    combined_df['tp'] = (combined_df['tp'] * 10).astype(int)
    combined_df['cb'] = (combined_df['cb'] * 10).astype(int)

    # 4) 컬럼 순서
    metric_cols = ["Sharpe", "Sortino", "MaxDrawdown",
                   "AnnualReturn", "AnnualVol",
                   "FinalWealthFactor", "WinRate"]

    param_cols = ["models", "dp", "sl", "pl", "dm", "nh", "nl",
                  "si", "sc", "hc", "mp", "ff", "tp", "cb", "itr"] 

    combined_df = combined_df[param_cols + metric_cols]


In [2]:
mean_data = combined_df.groupby(["models","dp","sl","pl","dm","nh","nl","si","hc", "mp", "ff","tp"]).mean().round(4)
std_data = combined_df.groupby(["models","dp","sl","pl","dm","nh","nl","si","hc", "mp", "ff","tp"]).std().round(4)

In [3]:
mean_data = mean_data.reset_index()
std_data = std_data.reset_index()

In [4]:
mean_data["dp"].unique()
pd.set_option('display.max_columns', 100)

In [5]:
mean_data[mean_data["dp"] == 30].sort_values("Sharpe", ascending= False).head(1)

,models,dp,sl,pl,dm,nh,nl,si,hc,mp,ff,tp,sc,cb,itr,Sharpe,Sortino,MaxDrawdown,AnnualReturn,AnnualVol,FinalWealthFactor,WinRate
0,Signature_FULL,30,60,20,8,8,1,2,64,5,64,13,1.0,0.0,1.0,0.5496,0.6797,0.3415,0.0943,0.2004,1.5678,0.5275


In [6]:
std_data[(std_data["dp"] == 30) &
         (std_data["dm"] == 8) &
         (std_data["nh"] == 8) &
         (std_data["nl"] == 1) &
         (std_data["hc"] == 64) &
         (std_data["ff"] == 64)][["dp","Sharpe","Sortino","MaxDrawdown","FinalWealthFactor"]].round(4)

,dp,Sharpe,Sortino,MaxDrawdown,FinalWealthFactor
0,30,0.0552,0.0727,0.0162,0.0973


In [7]:
mean_data[mean_data["dp"] == 40].sort_values("Sharpe", ascending= False).head(1)

,models,dp,sl,pl,dm,nh,nl,si,hc,mp,ff,tp,sc,cb,itr,Sharpe,Sortino,MaxDrawdown,AnnualReturn,AnnualVol,FinalWealthFactor,WinRate
1,Signature_FULL,40,60,20,8,4,2,2,8,5,32,13,1.0,0.0,1.0,0.6717,0.8232,0.3611,0.1239,0.2057,1.7903,0.5341


In [8]:
std_data[(std_data["dp"] == 40) &
         (std_data["dm"] == 8) &
         (std_data["nh"] == 4) &
         (std_data["nl"] == 2) &
         (std_data["hc"] == 8) &
         (std_data["ff"] == 32)][["dp","Sharpe","Sortino","MaxDrawdown","FinalWealthFactor"]].round(4)

,dp,Sharpe,Sortino,MaxDrawdown,FinalWealthFactor
1,40,0.0628,0.0792,0.0037,0.1023


In [9]:
mean_data[mean_data["dp"] == 50].sort_values("Sharpe", ascending= False).head(1)

,models,dp,sl,pl,dm,nh,nl,si,hc,mp,ff,tp,sc,cb,itr,Sharpe,Sortino,MaxDrawdown,AnnualReturn,AnnualVol,FinalWealthFactor,WinRate
2,Signature_FULL,50,60,20,8,8,2,2,64,5,8,13,1.0,0.0,1.0,0.7715,0.9743,0.3271,0.1396,0.193,1.9215,0.5331


In [10]:
std_data[(std_data["dp"] == 50) &
         (std_data["dm"] == 8) &
         (std_data["nh"] == 8) &
         (std_data["nl"] == 2) &
         (std_data["hc"] == 64) &
         (std_data["ff"] == 8)][["dp","Sharpe","Sortino","MaxDrawdown","FinalWealthFactor"]].round(4)

,dp,Sharpe,Sortino,MaxDrawdown,FinalWealthFactor
2,50,0.0627,0.0998,0.0094,0.1795


In [11]:
# BASE SETTING
mean_data[(mean_data["dm"] == 8) &
         (mean_data["nh"] == 8) &
         (mean_data["nl"] == 2) &
         (mean_data["hc"] == 64) &
         (mean_data["ff"] == 8)][["dp","Sharpe","Sortino","MaxDrawdown","FinalWealthFactor"]].round(4)

,dp,Sharpe,Sortino,MaxDrawdown,FinalWealthFactor
2,50,0.7715,0.9743,0.3271,1.9215


In [12]:
# BASE SETTING
std_data[(std_data["dm"] == 8) &
         (std_data["nh"] == 8) &
         (std_data["nl"] == 2) &
         (std_data["hc"] == 64) &
         (std_data["ff"] == 8)][["dp","Sharpe","Sortino","MaxDrawdown","FinalWealthFactor"]].round(4)

,dp,Sharpe,Sortino,MaxDrawdown,FinalWealthFactor
2,50,0.0627,0.0998,0.0094,0.1795
